#### Steps
1. Import Modules
2. Load ResNet18 without weights and change last layer to accomodate change in classes
3. Create custom dataset classes
4. Extract Labels
5. Creating train_loader

In [1]:
import numpy as np
import time
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
from torchsummary import summary
import torch.nn as nn
import torch
from torch.autograd.variable import Variable
from torchvision import datasets, models, transforms
import math
import torch.utils.model_zoo as model_zoo
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import rasterio

In [1]:
# loading ResNet Model
model = models.resnet18(pretrained=False)

# Changing last layer
num_final_in = model.fc.in_features

# The final layer of the model is model.fc so we can basically just overwrite it 
# to have the output = number of classes we need. Say, 300 classes.
NUM_CLASSES = 3
model.fc = nn.Linear(num_final_in, NUM_CLASSES)

NameError: name 'models' is not defined

In [5]:
counter = 0
for child in model.children():
    #print(child)
    print("--", counter,'----------', child)
    counter += 1

-- 0 ---------- Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
-- 1 ---------- BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
-- 2 ---------- ReLU(inplace)
-- 3 ---------- MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
-- 4 ---------- Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(6

In [6]:
summary(model, (13,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]          40,768
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [4]:
# Creating custom Dataset classes

class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        if self.transform:
            x = self.transform(x)    
        return x, y
    
    def __len__(self):
        return len(self.data)

In [5]:
#Extracting Labels
df=pd.read_csv(r"C:\Users\AJain7\OneDrive - Stryker\Courses\Satellite Project\Village_Labels.csv")  # --> Contain village label
village_code=df["Town/Village"].values
emp_label=df["Village_HHD_Cluster_MSW"].values
actual_labels= [ int(c) for c in emp_label]
s1 = pd.Series(actual_labels,index=list(village_code))

def slice_x(img,resize_dim_x):
    x,y,_ = img.shape
    startx = x//2-(resize_dim_x//2)
    return img[startx:startx+resize_dim_x, :,:]

def slice_y(img,resize_dim_y):
    x,y,_ = img.shape
    starty = y//2-(resize_dim_y//2)
    return img[:,starty:starty+resize_dim_y,:]

def crop_center(img,cropx,cropy): #Function for cropping the image from the center
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)
    return img[starty:starty+cropy,startx:startx+cropx]

all_img = []
all_label = np.array([])
resizeDim = 224
nchannels = 13
path = r"C:\Users\AJain7\Downloads\sat_images"  # --> Contains Satellite images

for file in os.listdir(path):
    filename = os.path.join(path,file)
    dataset = rasterio.open(filename)
    village_code = int(file.split('@')[3].split('.')[0])
    label = s1.loc[village_code]
    all_label = np.append(all_label,label)
    
    #X=np.array([]).reshape((0,resizeDim,resizeDim, nchannels))
    band1 = dataset.read(1)
    band2 = dataset.read(2)
    band3 = dataset.read(3)
    band4 = dataset.read(4)
    band5 = dataset.read(5)
    band6 = dataset.read(6)
    band7 = dataset.read(7)
    band8 = dataset.read(8)
    band9 = dataset.read(9)
    band10 = dataset.read(10)
    band11 = dataset.read(11)
    band12 = dataset.read(12)
    band13 = dataset.read(13)
    cd = np.dstack((band1, band2, band3, band4, band5, band6, band7, band8, band9, band10, band11, band12, band13))
    #print(cd.shape)
    
    if (cd.shape[0] > resizeDim or cd.shape[1] > resizeDim):
        if(cd.shape[0] > resizeDim and cd.shape[1] > resizeDim):
            combinedData = slice_x(cd,resize_dim_x=resizeDim)
            combinedData = slice_y(combinedData, resize_dim_y=resizeDim)
        elif(cd.shape[0] > resizeDim and cd.shape[1] <= resizeDim):
            combinedData = slice_x(cd,resize_dim_x=resizeDim)
        elif(cd.shape[0] <= resizeDim and cd.shape[1] > resizeDim):
            combinedData = slice_y(cd, resize_dim_y=resizeDim)
    else:
        combinedData = cd
    
    left = (resizeDim-combinedData.shape[0])//2
    right = resizeDim-combinedData.shape[0] - left
    up = (resizeDim-combinedData.shape[1])//2
    down = resizeDim-combinedData.shape[1] - up
    
    data = np.lib.pad(combinedData, [(left,right),(up,down),(0,0)], 'constant')
    data = np.reshape(data,(1,nchannels,resizeDim,resizeDim))
    all_img.append(data)

ai = np.vstack(all_img)

# ai --> All images of numpy array
# all_label --> Corresponding labels

In [10]:
dataset = MyDataset(ai, all_label)

print('Number of Samples:',dataset.__len__())
print(dataset.__getitem__(9)[1])

#loader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available() )
train_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)

Number of Samples: 63
tensor(2)


In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    
    for epoch in range(num_epochs):
        
        scheduler.step()
        model.train()
        
        for i, (images,labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            #Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()    
            
            print('Epoch [{}/{}],  Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, 100, loss.item()))

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# loss Function and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
els = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [8]:
model = train_model(model, criterion, optimizer_ft, els, num_epochs=3)

Epoch [1/3],  Step [1/100], Loss: 1.1070
Epoch [1/3],  Step [2/100], Loss: 0.9407
Epoch [1/3],  Step [3/100], Loss: 0.9639
Epoch [1/3],  Step [4/100], Loss: 0.9112
Epoch [2/3],  Step [1/100], Loss: 0.7630
Epoch [2/3],  Step [2/100], Loss: 0.6448
Epoch [2/3],  Step [3/100], Loss: 0.7975
Epoch [2/3],  Step [4/100], Loss: 0.9290
Epoch [3/3],  Step [1/100], Loss: 0.6867
Epoch [3/3],  Step [2/100], Loss: 0.5526
Epoch [3/3],  Step [3/100], Loss: 0.4984
Epoch [3/3],  Step [4/100], Loss: 0.6677
